In [1]:
#pytorch lib
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms,models
from torch.utils.data import Dataset,random_split
import torch.optim as optim


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os,shutil,warnings
import glob
import random 
import rasterio
import json

from tqdm import tqdm
from PIL import Image
from IPython.display import display
from torchvision import transforms
from typing import Tuple, Dict, List

from sklearn.metrics import precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay


random.seed(69)


In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


#initialized dataset
root_dir = 'datasets/EuroSATallBands/'
dataset = EuroSATMultiBandDataset(root_dir, transform=transform)


train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


NameError: name 'EuroSATMultiBandDataset' is not defined

In [8]:
model = models.convnext_tiny(weights='DEFAULT')
model.features[0][0] = nn.Conv2d(6, 96, kernel_size=(4, 4), stride=(4, 4))
model.classifier[2] = nn.Linear(model.classifier[2].in_features,10)


model_path = 'weights/convnext_model_epoch_20.pth'

model.load_state_dict(torch.load(model_path))
model.to(device)

model.eval()  # Set the model to evaluation mode
model.to(device)
val_loss = 0.0
correct = 0
total = 0
all_labels = []
all_predictions = []
with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc='Evaluating', leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

accuracy = 100 * correct / total
precision = precision_score(all_labels, all_predictions, average='macro')
recall = recall_score(all_labels, all_predictions, average='macro')
cm = confusion_matrix(all_labels, all_predictions)

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(10))
disp.plot(cmap='Blues')


NameError: name 'val_loader' is not defined